In [1]:
# Imports
import pandas as pd
import time
import requests
import re

from bs4 import BeautifulSoup
#from lxml import html

In [2]:
# Setting base url
url = 'https://www.sailorsongs.com/'

In [3]:
# Requesting links page
res_links = requests.get(f'{url}lyrics.html')
res_links.status_code

200

In [4]:
# Converting requests result into list of links
links_soup = BeautifulSoup(res_links.content, 'lxml')
links_list = links_soup.find_all('a', href=True)
links_list = [x['href'] for x in links_list]

In [5]:
# Dropping non-song links from list
del links_list[:7]
del links_list[-6:]

In [6]:
# Creating empty dictionary to hold songs
shanties_dict = {}

In [7]:
# Iterating through
for link in links_list:
    # Requesting lyrics page
    res_lyrics = requests.get(f'{url}{link}')
    
    # Putting in a small sleep to limit requests on server to under 100 per minutes
    time.sleep(.75)
    print(f'{res_lyrics.status_code} {link}')
    
    # Pulling song title
    title = BeautifulSoup(res_lyrics.text).title.text

    # Converting lyrics request into list
    lyrics_soup = BeautifulSoup(res_lyrics.content, 'lxml')
    lyrics_raw = lyrics_soup.find_all('p')
    # Checking to see if lyrics block has intro text
    if lyrics_raw[0].text.find('www.SailorSongs.com') != -1:
        del lyrics_raw[0]
    # Deleting non-lyrics from list
    del lyrics_raw[-1]

    # Converting soup into list
    lyrics_raw = [x.text for x in lyrics_raw]
    
    # Converting list of strings to single string
    lyrics = ''
    lyrics = lyrics.join(lyrics_raw)
    
    # Updating Shanties Dict
    shanties_dict.update({title : lyrics})
'''
Commenting out to see if cleaning all lyrics in one file is more efficient
    # Removing unwanted character strings
    lyrics = [x.replace("\n", '') for x in lyrics]
    lyrics = [x.replace("\xa0", '') for x in lyrics]
    lyrics = [x.replace("courtesy of www.SailorSongs.com", ' ') for x in lyrics]

    # Removing multiple spaces from lyrics
    lyrics = ' '.join(lyrics)
    lyrics = ' '.join(lyrics.split())
    
'''

    

200 a_cruising_we_will_go.html
200 a_hundred_years_ago.html
200 a_life_on_the_ocean_wave.html
200 a_long_time_ago.html
200 a_rollin_down_the_river.html
200 a_rovin.html
200 a_sailor_loves.html
200 a_wet_sheet_and_a_flowing_sea.html
200 abdul_abulbul_amir.html
200 across_the_western_ocean.html
200 admiral_benbow.html
200 aha_me_boys_lukeys_boat.html
200 alabama_john_cherokee.html
200 all_for_me_grog.html
200 anchors_aweigh_navy_song .html
200 andrew_barton.html
200 around_cape_horn.html
200 asleep_in_the_deep.html
200 away_rio.html
200 aweigh_santy_ano.html
200 the_ballad_of_captain_kidd.html
200 the_balena.html
200 baltimore_shanty.html
200 banana_boat_song .html
200 bang_lulu.html
200 banks_of_the_brandywine.html
200 the_banks_of_new_foundland.html
200 the_banks_of_the_nile.html
200 the_banks_of_the_sacramento.html
200 the_banks_of_sweet_loch_ray.html
200 the_bark_gay_head.html
200 barnacle_bill_the_sailor.html
200 battle_of_the_chesapeake_and_the_shannon.html
200 bay_of_biscay.html
2

'\nCommenting out to see if cleaning all lyrics in one file is more efficient\n    # Removing unwanted character strings\n    lyrics = [x.replace("\n", \'\') for x in lyrics]\n    lyrics = [x.replace("\xa0", \'\') for x in lyrics]\n    lyrics = [x.replace("courtesy of www.SailorSongs.com", \' \') for x in lyrics]\n\n    # Removing multiple spaces from lyrics\n    lyrics = \' \'.join(lyrics)\n    lyrics = \' \'.join(lyrics.split())\n    \n'

In [8]:
# Creating data frame for songs and lyrics
sailor_songs_songs = pd.DataFrame(shanties_dict.items(), columns=['title', 'lyrics'])

In [10]:
# Writing to csv for use in other scripts
sailor_songs_songs.to_csv('../data/sailor_songs_songs.csv')